<a href="https://colab.research.google.com/github/mrunallll/QnA-using-Transformers/blob/main/BERT_QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Custom QnA using cdQA - Closed Domain Question Answering

In [ ]:
#Installing the cdqa package
!pip install cdqa

In [1]:
import os 
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)



Downloading bert model, which is pretrained on SQUAD 1.1 Dataset

In [2]:
download_model(model='bert-squad_1.1', dir='./models')

In [3]:
!mkdir docs

#Storing a storybook PDF from web 
!wget -P ./docs/ http://www.lcpese.it/assets/Uploads/3134.pdf

--2021-07-21 08:00:17--  http://www.lcpese.it/assets/Uploads/3134.pdf
Resolving www.lcpese.it (www.lcpese.it)... 91.142.143.143
Connecting to www.lcpese.it (www.lcpese.it)|91.142.143.143|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58775 (57K) [application/pdf]
Saving to: ‘./docs/3134.pdf’

3134.pdf            100%[===================>]  57.40K   189KB/s    in 0.3s    

2021-07-21 08:00:18 (189 KB/s) - ‘./docs/3134.pdf’ saved [58775/58775]



In [4]:
df1_storybook=pdf_converter('./docs/')
df1_storybook.head()

2021-07-21 08:11:34,569 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to /tmp/tika-server.jar.
2021-07-21 08:11:35,137 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to /tmp/tika-server.jar.md5.
2021-07-21 08:11:35,650 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


,title,paragraphs
0,Five Stories by Alphonse Daudet,"[Five Short Stories, Five Short Stories, Alpho..."
1,YOLO orignial paper,"[You Only Look Once:Unified, Real-Time Object ..."


Displaying the contents of the pdfs we have considered in the dataframe

#We are using two different files -
1. A storybook having fiction stories from Alphonse Daudet
2. A Deep Learning research paper 

Thus, the context of both the documents is very different, and the model should return appropriate answers to queries, without confusing.

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
df1_storybook.head()

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

#Since we are using pretrained BERT, we call a retriever method as the Reader is already pretrained on SQuAD 1.1 dataset

In [6]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)
cdqa_pipeline.fit_retriever(df=df1_storybook)

100%|██████████| 231508/231508 [00:00<00:00, 862589.96B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [7]:
# Sending a question to the pipeline and getting back predicted answer
query = 'What is the story about?'
prediction = cdqa_pipeline.predict(query) #give out three predictions

print('query: {}\n'.format(query))
print('answer: {}\n'.format(prediction[0]))
print('title of PDF: {}\n'.format(prediction[1]))
print('paragraph where ans was found:\n {}'.format(prediction[2]))

query: What is the story about?

answer: the siege of Paris

title of PDF: Five Stories by Alphonse Daudet

paragraph where ans was found:
 WE were going up Avenue des Champs-Elysées with Dr. V——, asking the shell-riddled walls, and thesidewalks torn up by grape-shot, for the story of the siege of Paris, when, just before we reached theRond-point de l’Etoile, the doctor stopped and, pointing to one of the great corner houses so proudlygrouped about the Arc de Triomphe, said to me:  “Do you see those four closed windows up there on that balcony? In the early days of August, thatterrible August of last year, so heavily laden with storms and disasters, I was called there to see a case ofapoplexy. It was the apartment of Colonel Jouve, a cuirassier of the First Empire, an old enthusiast on thesubject of glory and patriotism, who had come to live on the Champs-Élysées, in an apartment with abalcony, at the outbreak of the war. Guess why? In order to witness the triumphant return of our troo

In [9]:
query1="What did Monsieur Hamel teach?"
pred=cdqa_pipeline.predict(query1)

print('query: {}\n'.format(query1))
print('answer: {}\n'.format(pred[0]))
print('title of PDF: {}\n'.format(pred[1]))
print('paragraph where ans was found:\n {}'.format(pred[2]))

query: What did Monsieur Hamel teach?

answer: inhistory

title of PDF: Five Stories by Alphonse Daudet

paragraph where ans was found:
 I WAS very late for school that morning, and I was terribly afraid of being scolded, especially asMonsieur Hamel had told us that he should examine us on participles, and I did not know the first thingabout them. For a moment I thought of staying away from school and wandering about the fields. It wassuch a warm, lovely day. I could hear the blackbirds whistling on the edge of the wood, and in theRippert field, behind the sawmill, the Prussians going through their drill. All that was much moretempting to me than the rules concerning participles; but I had the strength to resist, and I ran as fast as Icould to school.  As I passed the mayor’s office, I saw that there were people gathered about the little board on whichnotices were posted. For two years all our bad news had come from that board—battles lost,conscriptions, orders from headquarters; and I

In [13]:
query1="What is unique about YOLO?"
pred=cdqa_pipeline.predict(query1)

print('query: {}\n'.format(query1))
print('answer: {}\n'.format(pred[0]))
print('title of PDF: {}\n'.format(pred[1]))
print('paragraph where ans was found:\n {}'.format(pred[2]))

query: What is unique about YOLO?

answer: fast, accurate object detector

title of PDF: YOLO orignial paper

paragraph where ans was found:
 YOLO is a fast, accurate object detector, making it idealfor computer vision applications. We connect YOLO to awebcam and verify that it maintains real-time performance,


In [16]:
query1="What is YOLO architecture about?"
pred=cdqa_pipeline.predict(query1)

print('query: {}\n'.format(query1))
print('answer: {}\n'.format(pred[0]))
print('title of PDF: {}\n'.format(pred[1]))
print('paragraph where ans was found:\n {}'.format(pred[2]))

query: What is YOLO architecture about?

answer: image classification

title of PDF: YOLO orignial paper

paragraph where ans was found:
 Our network architecture is inspired by the GoogLeNetmodel for image classification [34]. Our network has 24convolutional layers followed by 2 fully connected layers.Instead of the inception modules used by GoogLeNet, wesimply use 1× 1 reduction layers followed by 3× 3 convo-lutional layers, similar to Lin et al [22]. The full network isshown in Figure 3.


In [17]:
query1="What can YOLO do?"
pred=cdqa_pipeline.predict(query1)

print('query: {}\n'.format(query1))
print('answer: {}\n'.format(pred[0]))
print('title of PDF: {}\n'.format(pred[1]))
print('paragraph where ans was found:\n {}'.format(pred[2]))

query: What can YOLO do?

answer: rescore Fast R-CNN detections and reduce the er-rors from background false positives

title of PDF: YOLO orignial paper

paragraph where ans was found:
 4. ExperimentsFirst we compare YOLO with other real-time detectionsystems on PASCAL VOC 2007. To understand the differ-ences between YOLO and R-CNN variants we explore theerrors on VOC 2007 made by YOLO and Fast R-CNN, oneof the highest performing versions of R-CNN [14]. Basedon the different error profiles we show that YOLO can beused to rescore Fast R-CNN detections and reduce the er-rors from background false positives, giving a significantperformance boost. We also present VOC 2012 results andcompare mAP to current state-of-the-art methods. Finally,we show that YOLO generalizes to new domains better thanother detectors on two artwork datasets.


In [18]:
query1="How is YOLO different from other techniques?"
pred=cdqa_pipeline.predict(query1)

print('query: {}\n'.format(query1))
print('answer: {}\n'.format(pred[0]))
print('title of PDF: {}\n'.format(pred[1]))
print('paragraph where ans was found:\n {}'.format(pred[2]))

query: How is YOLO different from other techniques?

answer: preciselybecause YOLO makes different kinds of mistakes at testtime

title of PDF: YOLO orignial paper

paragraph where ans was found:
 The boost from YOLO is not simply a byproduct ofmodel ensembling since there is little benefit from combin-ing different versions of Fast R-CNN. Rather, it is preciselybecause YOLO makes different kinds of mistakes at testtime that it is so effective at boosting Fast R-CNN’s per-formance.
